In [2]:
import numpy as np
import pandas as pd
import optuna
from lightgbm import LGBMClassifier
from optuna import Trial
from optuna.samplers import TPESampler
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split

In [15]:
train = pd.read_excel("/Users/lhs/Desktop/GitHub/MLT/NLP/news_label.xlsx")
train = train.rename(columns={'dates':'date', 'titles':'title','contents':'content'})
train = train.drop(columns=['근거'])
train['content'] = train['title'] + train['content']
train = train.drop(columns=['title'])

test = pd.read_csv("/Users/lhs/Desktop/GitHub/MLT/NLP/news_label_x.csv")
test = test.rename(columns={'dates':'date', 'titles':'title','contents':'content'})
test = test.drop(columns=['label'])
test['content'] = test['title'] + test['content']
test = test.drop(columns=['title'])


In [20]:
train

,date,content,label
0,2021.10.05,[[톡톡 이상품] 한빛코리아 / 다지워 네이처 클렌징 솝]//[\n마스카라까지 지우...,1
1,2021.10.07,"[[격동의 유통가④]편의점 성장 정체…미래는 근거리 쇼핑]//[""파괴적 커머스 시대...",1
2,2021.10.08,[株머니 매니저의 HOT종목]//[\n\n\n\n\n 씨에스윈드는 풍력발전기를 구성...,1
3,2021.10.11,"[김민수 더맘마 대표 ""동네마트와 '상생'이 경쟁력…30분 총알배송 목표""]//[\...",1
4,2021.10.13,"[""카카오 보고있나""…진격의 네이버페이, 中 최대 직구몰 손잡았다]//[[주간투자동...",1
...,...,...,...
274,2022.09.23,"[WCP 흥행 실패에 컬리, 케이뱅크 상장 시기 '안갯속']//[\n심사 승인 났는...",0
275,2022.09.26,"[[먹거리+IT] 네이처랩 오경주 대표, “맛있고 건강한 주스 추천 서비스를 개발합...",1
276,2022.09.27,"[""투자 늘렸는데 이제와서…"" 퀵커머스마저 규제하나 '초긴장']//[\n■국감 오르...",0
277,2022.09.29,[국내 이커머스 상장 1호 주인공 누가될까 [격변의 이커머스①]]//[\n이커머스 ...,0


In [17]:
test

,date,content
0,2021.10.05,[[톡톡 이상품] 한빛코리아 / 다지워 네이처 클렌징 솝]//[\n마스카라까지 지우...
1,2021.10.07,"[[격동의 유통가④]편의점 성장 정체…미래는 근거리 쇼핑]//[""파괴적 커머스 시대..."
2,2021.10.08,[株머니 매니저의 HOT종목]//[\n\n\n\n\n 씨에스윈드는 풍력발전기를 구성...
3,2021.10.11,"[김민수 더맘마 대표 ""동네마트와 '상생'이 경쟁력…30분 총알배송 목표""]//[\..."
4,2021.10.13,"[""카카오 보고있나""…진격의 네이버페이, 中 최대 직구몰 손잡았다]//[[주간투자동..."
...,...,...
274,2022.09.23,"[WCP 흥행 실패에 컬리, 케이뱅크 상장 시기 '안갯속']//[\n심사 승인 났는..."
275,2022.09.26,"[[먹거리+IT] 네이처랩 오경주 대표, “맛있고 건강한 주스 추천 서비스를 개발합..."
276,2022.09.27,"[""투자 늘렸는데 이제와서…"" 퀵커머스마저 규제하나 '초긴장']//[\n■국감 오르..."
277,2022.09.29,[국내 이커머스 상장 1호 주인공 누가될까 [격변의 이커머스①]]//[\n이커머스 ...


In [25]:
train_ohe = pd.get_dummies(train)
test_ohe = pd.get_dummies(test)

In [29]:
X = train_ohe.drop(['label'], axis = 1)
y = train['label']
X_test = test_ohe.copy()

In [30]:
X

date_2021.10.05  date_2021.10.07  date_2021.10.08  date_2021.10.11  \
0                  1                0                0                0   
1                  0                1                0                0   
2                  0                0                1                0   
3                  0                0                0                1   
4                  0                0                0                0   
..               ...              ...              ...              ...   
274                0                0                0                0   
275                0                0                0                0   
276                0                0                0                0   
277                0                0                0                0   
278                0                0                0                0   

     date_2021.10.13  date_2021.10.14  date_2021.10.15  date_2021.10.18  \
0                  0                0                0                0   
1                  0                0                0                0   
2                  0                0                0                0   
3                  0                0                0                0   
4                  1                0                0                0   
..               ...              ...              ...              ...   
274                0                0                0                0   
275                0                0                0                0   
276                0                0                0                0   
277                0                0                0                0   
278                0                0                0                0   

     date_2021.10.19  date_2021.10.20  ...  \
0                  0                0  ...   
1                  0                0  ...   
2                  0                0  ...   
3                  0                0  ...   
4                  0                0  ...   
..               ...              ...  ...   
274                0                0  ...   
275                0                0  ...   
276                0                0  ...   
277                0                0  ...   
278                0                0  ...   

     content_[하반기 IPO 대어 줄 섰는데…증권사는 주관만 급급]//[\nLG에너지솔루션, 상장 당일 1000건 넘는 민원증권사는 거래소 탓, 거래소는 '정상 처리' 주장전산비, 전년보다 14% 늘어도…자본총계 1% 내외하반기 컬리·CJ올리브영 등 대기…투자자 불만 재점화 우려[이데일리 김인경 기자] 30대 박모 씨는 올해 초 인생 처음으로 공모주 투자에 나서기로 마음을 먹었다. LG에너지솔루션(373220)이라면 공모를 해볼 만하다는 생각이 들었기 때문이다. 박씨는 유투브를 보며 공부를 한 끝에 가장 많은 주식을 받을 수 있을 것 같은 주관사를 골라 계좌를 개설하고 증거금을 냈다. 그 결과 2주를 받았다. 1월 27일 오전 9시가 되자마자 박씨는 LG에너지솔루션에 매도를 걸었지만 모바일트레이딩시스템(MTS)은 다음 화면으로 넘어가지 않았다. 잠시만 기다려달라는 창만 뜰 뿐이었다. 박씨는 결국 57만원에 매도를 하려고 했지만, 그보다 5만원 낮은 52만원에 팔 수밖에 없었다. IPO마다 잇딴 접속지연…“연례행사”5일 금융투자업계에 따르면 LG에너지솔루션 상장일 전산장애 관련 금감원에 접수된 민원 건수는 1019건에 달했다. 이 중 22건은 자율조정과 민원인의 철회로 해결됐지만, 997건은 여전히 해결점을 찾지 못하고 있다. 이들 민원은 LG에너지솔루션이 상장한 지난 1월 27일 개장 직후 공모주 투자자들의 주문이 한꺼번에 몰면서 증권사 홈트레이딩시스템(HTS)과 MTS에서 전산 장애가 발생해 제때 매도를 하지 못했다는 내용이다. 하지만 증권사은 “전산 시스템에 문제가 없었다”며 보상을 해줄 수 없다는 입장이다. 대표주관사였던 KB증권은 “상장일 타사대체출고 지연, 주문처리지연 등은 KB증권의 시스템의 장애가 아니며, 자체의 내부적인 문제로 발생한 사항도 아니다”라면서 “이는 타사에서도 동일하게 발생한 사항으로, 대외 유관기관의 처리 지연에 발생한 문제”라고 선을 그었다. 보통 투자자들이 주문을 넣게 되면 증권사를 거쳐 한국거래소 시스템으로 주문 회선이 진행되는 만큼, 거래소 측의 문제라는 게 증권사의 입장이다. 하지만 거래소도 책임을 외면하고 있다. 거래소는 “주문이 폭증하면서 시장의 호가 접수, 체결 및 결과 송신 과정이 평소보다 다소 길어진 것은 맞다”면서도 “오류, 착오 없이 접수한 호가 순서대로 정상적으로 처리됐다”고 말했다. 거래소는 처리용량과 속도를 개선하는 차세대 시스템을 개발해 내년 가동할 계획이다.   \n\n\n\n[그래픽=이데일리 문승용 기자]MTS 문제에 증권업계만 민원 역주행금융투자업계에서는 이같은 갈등이 LG에너지솔루션만의 문제는 아니라고 지적한다. 증권사는 IPO 주관사 전쟁에, 거래소는 대어 상장에 혈안이 돼 있으면서 투자자들의 접근성 보장엔 소홀하다 보니 이런 문제가 정기적으로 나타난다는 것이다. 실제 대규모 IPO 주관사 프리젠테이션에는 개별 증권사의 기업금융부문 부문장은 물론, 최고경영자(CEO)급까지 직접 출동하는 반면, 여전히 정보기술(IT) 투자에는 소극적이다. 지난해 59개 증권사의 전산운용비가 전년보다 14.91% 증가한 6667억5671만원으로 나타났지만, 개별 증권사의 자산 총계를 뜯어보면 전산운용비는 1%대에 불과하다. 한 중형증권사 관계자는 “IT 인력의 비용이 천정부지로 높아진 데다 한 증권사가 초대형 IPO를 맡는 경우는 많아야 1년에

In [24]:
train_ohe

label  date_2021.10.05  date_2021.10.07  date_2021.10.08  \
0        1                1                0                0   
1        1                0                1                0   
2        1                0                0                1   
3        1                0                0                0   
4        1                0                0                0   
..     ...              ...              ...              ...   
274      0                0                0                0   
275      1                0                0                0   
276      0                0                0                0   
277      0                0                0                0   
278      1                0                0                0   

     date_2021.10.11  date_2021.10.13  date_2021.10.14  date_2021.10.15  \
0                  0                0                0                0   
1                  0                0                0                0   
2                  0                0                0                0   
3                  1                0                0                0   
4                  0                1                0                0   
..               ...              ...              ...              ...   
274                0                0                0                0   
275                0                0                0                0   
276                0                0                0                0   
277                0                0                0                0   
278                0                0                0                0   

     date_2021.10.18  date_2021.10.19  ...  \
0                  0                0  ...   
1                  0                0  ...   
2                  0                0  ...   
3                  0                0  ...   
4                  0                0  ...   
..               ...              ...  ...   
274                0                0  ...   
275                0                0  ...   
276                0                0  ...   
277                0                0  ...   
278                0                0  ...   

     content_[하반기 IPO 대어 줄 섰는데…증권사는 주관만 급급]//[\nLG에너지솔루션, 상장 당일 1000건 넘는 민원증권사는 거래소 탓, 거래소는 '정상 처리' 주장전산비, 전년보다 14% 늘어도…자본총계 1% 내외하반기 컬리·CJ올리브영 등 대기…투자자 불만 재점화 우려[이데일리 김인경 기자] 30대 박모 씨는 올해 초 인생 처음으로 공모주 투자에 나서기로 마음을 먹었다. LG에너지솔루션(373220)이라면 공모를 해볼 만하다는 생각이 들었기 때문이다. 박씨는 유투브를 보며 공부를 한 끝에 가장 많은 주식을 받을 수 있을 것 같은 주관사를 골라 계좌를 개설하고 증거금을 냈다. 그 결과 2주를 받았다. 1월 27일 오전 9시가 되자마자 박씨는 LG에너지솔루션에 매도를 걸었지만 모바일트레이딩시스템(MTS)은 다음 화면으로 넘어가지 않았다. 잠시만 기다려달라는 창만 뜰 뿐이었다. 박씨는 결국 57만원에 매도를 하려고 했지만, 그보다 5만원 낮은 52만원에 팔 수밖에 없었다. IPO마다 잇딴 접속지연…“연례행사”5일 금융투자업계에 따르면 LG에너지솔루션 상장일 전산장애 관련 금감원에 접수된 민원 건수는 1019건에 달했다. 이 중 22건은 자율조정과 민원인의 철회로 해결됐지만, 997건은 여전히 해결점을 찾지 못하고 있다. 이들 민원은 LG에너지솔루션이 상장한 지난 1월 27일 개장 직후 공모주 투자자들의 주문이 한꺼번에 몰면서 증권사 홈트레이딩시스템(HTS)과 MTS에서 전산 장애가 발생해 제때 매도를 하지 못했다는 내용이다. 하지만 증권사은 “전산 시스템에 문제가 없었다”며 보상을 해줄 수 없다는 입장이다. 대표주관사였던 KB증권은 “상장일 타사대체출고 지연, 주문처리지연 등은 KB증권의 시스템의 장애가 아니며, 자체의 내부적인 문제로 발생한 사항도 아니다”라면서 “이는 타사에서도 동일하게 발생한 사항으로, 대외 유관기관의 처리 지연에 발생한 문제”라고 선을 그었다. 보통 투자자들이 주문을 넣게 되면 증권사를 거쳐 한국거래소 시스템으로 주문 회선이 진행되는 만큼, 거래소 측의 문제라는 게 증권사의 입장이다. 하지만 거래소도 책임을 외면하고 있다. 거래소는 “주문이 폭증하면서 시장의 호가 접수, 체결 및 결과 송신 과정이 평소보다 다소 길어진 것은 맞다”면서도 “오류, 착오 없이 접수한 호가 순서대로 정상적으로 처리됐다”고 말했다. 거래소는 처리용량과 속도를 개선하는 차세대 시스템을 개발해 내년 가동할 계획이다.   \n\n\n\n[그래픽=이데일리 문승용 기자]MTS 문제에 증권업계만 민원 역주행금융투자업계에서는 이같은 갈등이 LG에너지솔루션만의 문제는 아니라고 지적한다. 증권사는 IPO 주관사 전쟁에, 거래소는 대어 상장에 혈안이 돼 있으면서 투자자들의 접근성 보장엔 소홀하다 보니 이런 문제가 정기적으로 나타난다는 것이다. 실제 대규모 IPO 주관사 프리젠테이션에는 개별 증권사의 기업금융부문 부문장은 물론, 최고경영자(CEO)급까지 직접 출동하는 반면, 여전히 정보기술(IT) 투자에는 소극적이다. 지난해 59개 증권사의 전산운용비가 전년보다 14.91% 증가한 6667억5671만원으로 나타났지만, 개별 증권사의 자산 총계를 뜯어보면 전산운용비는 1%대에 불과하다. 한 중형증권사 관계자는 “IT 인력의 비용이 천정부지로 높아진 데다 한 증권사가 초대형 IPO를 맡는 경우는 많아야 1년에 1~2번인데 이를 대비해서 대규모 예산을 편입하기는 쉽지 않다”라면서 “그러면서도 공모주 청약을 위한 신규 계좌 개설을 기대하는 것은 모순적인 태도”라고 꼬집었다. 금융감독원에 직접 접수된 민원을 대상으로 하는 ‘

In [34]:
import optuna 
from optuna import Trial, visualization
from optuna.samplers import TPESampler

from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

In [38]:
def objective(trial: Trial) -> float:
    params_lgb = {
        "random_state": 42,
        "verbosity": -1,
        "learning_rate": 0.05,
        "n_estimators": 10000,
        "objective": "multiclass",
        "metric": "multi_logloss",
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 3e-5),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 9e-2),
        "max_depth": trial.suggest_int("max_depth", 1, 20),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.4, 1.0),
        "subsample": trial.suggest_float("subsample", 0.3, 1.0),
        "subsample_freq": trial.suggest_int("subsample_freq", 1, 10),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "max_bin": trial.suggest_int("max_bin", 200, 500),
    }
    
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)

    model = LGBMClassifier(**params_lgb)
    model.fit(
        X_train,
        y_train,
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
        early_stopping_rounds=100,
        verbose=False,
    )

    lgb_pred = model.predict_proba(X_valid)
    log_score = log_loss(y_valid, lgb_pred)
    
    return log_score

In [39]:
sampler = TPESampler(seed=42)
study = optuna.create_study(
    study_name="lgbm_parameter_opt",
    direction="minimize",
    sampler=sampler,
)
study.optimize(objective, n_trials=10)
print("Best Score:", study.best_value)
print("Best trial:", study.best_trial.params)

[I 2022-10-11 16:33:08,922] A new study created in memory with name: lgbm_parameter_opt
/Users/lhs/miniforge3/envs/lhs/lib/python3.8/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/lhs/miniforge3/envs/lhs/lib/python3.8/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
[LightGBM] [Fatal] Number of classes should be specified and greater than 1 for multiclass training
[W 2022-10-11 16:33:08,934] Trial 0 failed because of the following error: L

LightGBMError: Number of classes should be specified and greater than 1 for multiclass training

In [36]:
# Console 출력 예)
# Best trial: score 0.6918131483269861,
# params {'n_estimators': 1988, 'max_depth': 15, 'min_child_weight': 171, 'gamma': 2, 'colsample_bytree': 0.7, 'lambda': 0.009025835256777458, 'alpha': 1.0684343216052703, 'subsample': 0.6} 